## XML Web Scraping Roll Call Data

In [471]:
#import libs
import urllib
import xml.etree.ElementTree as ET
import re
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from time import sleep

In [272]:
#test
#request
url = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1142/vote_114_2_00081.xml"
req = urllib.request.Request(url)
with urllib.request.urlopen(req) as response:
    the_page = response.read()

In [457]:
#begin parse with root
root = ET.fromstring(the_page)

In [454]:
#checkout children
for child in root:
    print(child.tag, ":", child.text)

congress : 114
session : 2
congress_year : 2016
vote_number : 163
vote_date : December 10, 2016,  12:57 AM
modify_date : December 10, 2016,  01:26 AM
vote_question_text : On the Motion (Motion to Concur in the House Amendment to S. 612)
vote_document_text : A bill to designate the Federal building and United States courthouse located at 1300 Victoria Street in Laredo, Texas, as the "George P. Kazen Federal Building and United States Courthouse".
vote_result_text : Motion Agreed to (78-21)
question : On the Motion
vote_title : Motion to Concur in the House Amendment to S. 612
majority_requirement : 1/2
vote_result : Motion Agreed to
document : 
    
amendment : 
    
count : 
    
tie_breaker : 
    
members : 
    


In [463]:
#bill_info
info={}
for key, value in master_roots_1.items():
    bill_info={}
    for child in value:
        bill_info[child.tag] = child.text
    info[key] = bill_info   
    

In [468]:
info_2 = {}
for key, value in master_roots_2.items():
    bill_info={}
    for child in value:
        bill_info[child.tag] = child.text
    info_2[str(int(key)+339)] = bill_info 

In [183]:
#parse members
member_names = []
parties = []
states = []
votes = []
ids = []
for element in root[-1]:
    member_names.append(element[1].text + ' '+ element[2].text)
    parties.append(element[3].text)
    states.append(element[4].text)
    votes.append(element[5].text)
    ids.append(element[6].text)

In [184]:
member_data = pd.DataFrame.from_dict({'member': member_names, 'party': parties, 'state': states, 'vote': votes, 'idx': ids})

In [185]:
member_data.T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
idx,S289,S340,S354,S317,S330,S341,S342,S370,S343,S223,...,S303,S384,S351,S326,S299,S327,S366,S316,S318,S247
member,Alexander Lamar,Ayotte Kelly,Baldwin Tammy,Barrasso John,Bennet Michael,Blumenthal Richard,Blunt Roy,Booker Cory,Boozman John,Boxer Barbara,...,Thune John,Tillis Thomas,Toomey Pat,Udall Tom,Vitter David,Warner Mark,Warren Elizabeth,Whitehouse Sheldon,Wicker Roger,Wyden Ron
party,R,R,D,R,D,D,R,D,R,D,...,R,R,R,D,R,D,D,D,R,D
state,TN,NH,WI,WY,CO,CT,MO,NJ,AR,CA,...,SD,NC,PA,NM,LA,VA,MA,RI,MS,OR
vote,Nay,Nay,Yea,Nay,Nay,Yea,Nay,Yea,Nay,Not Voting,...,Nay,Nay,Nay,Yea,Nay,Yea,Yea,Yea,Nay,Yea


In [189]:
def get_votes(root):
    votes = []
    for element in root[-1]:
        votes.append(element[5].text)
    return votes
def get_member_id(root):
    ids = []
    for element in root[-1]:
        ids.append(element[6].text)
    

In [196]:
def padded(num):
    l = len(str(num))
    zeros = "0"*(5-l)
    new = zeros+str(num)
    return new

In [ ]:
#create master dict of roots for each bill

In [205]:
#repeat the above process, but create a master dict of all roots
# this is a test, full script below
master_roots = {}
for i in range(1,3):
    n= padded(i)
    url = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1141/vote_114_1_"+ n + ".xml"
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
    root = ET.fromstring(the_page)
    master_roots[n] = root

In [206]:
#it works
master_roots

{'00001': <Element 'roll_call_vote' at 0x000001F541D0F7C8>,
 '00002': <Element 'roll_call_vote' at 0x000001F541D49548>}

In [230]:
#it works
for i in range(1,164):
    n = padded(i)
    #print (master_roots[n][-1][0][0].text)

### Create a dict of all the roll call votes

In [301]:
#session 1
master_roots_1 = {}
for i in range(1,340):
    n= padded(i)
    url = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1141/vote_114_1_"+ n + ".xml"
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
    root = ET.fromstring(the_page)
    master_roots_1[n] = root

In [519]:
#session 2
master_roots_2 = {}
for i in range(1,164):
    try:
        n= padded(i)
        url = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1142/vote_114_2_"+ n + ".xml"
        req = urllib.request.Request(url)
        with urllib.request.urlopen(req) as response:
            the_page = response.read()
        root = ET.fromstring(the_page)
        master_roots_2[n] = root
        sleep(2)
    except:
        continue

In [404]:
# make a list of the senators
senators = []
for i in range(100):
    senator = master_roots['00001'][-1][i][0].text
    senators.append(senator)

### Create votes table

In [406]:
# make a dict of all the votes for each bill
# first session
votes_dict_1 = {}
for key, value in master_roots_1.items():
    votes_dict_1[key] = get_votes(value)
    
#second session    
votes_dict_2 = {}
for key, value in master_roots_2.items():
    votes_dict_2[str(int(key)+339)] = get_votes(value)

In [408]:
#make the df
bills_df = pd.DataFrame.from_dict(votes_dict_1).join(pd.DataFrame.from_dict(votes_dict_2)).T
bills_df.columns = senators
bills_df.index.name = 'vote_number'
print (bills_df.shape)

#### Get outcomes

In [413]:
def get_outcome(root):
    return root[-6].text

In [495]:
#make a dict of all outcomes
outcomes_dict_1 = {}
for key, value in master_roots_1.items():
    outcomes_dict_1[key] = get_outcome(value)
outcomes_dict_2 = {}
for key, value in master_roots_2.items():
    outcomes_dict_2[str(int(key)+339)] = get_outcome(value)

In [496]:
#combine the dicts
for key, value in outcomes_dict_2.items():
    outcomes_dict_1[key] = value

In [611]:
#turn it into a series
outcomes = pd.Series(outcomes_dict_1)
outcomes.name = 'outcome'

In [613]:
#join the frames
votes = bills_df.join(outcomes)
new_index = [str(int(x)) for x in votes.index.values]
votes.index = new_index

In [620]:
#save
votes.to_csv('votes.csv')

In [624]:
#do some replacements
df = votes.replace({'Nay': 0, 'Yea': 1, 'Not Voting': 0, 'Present': 1})

In [627]:
#make a replacements dicts
outcome_replacements = {'Amendment Rejected': 0,
 'Bill Passed': 1,
 'Cloture on the Motion to Proceed Agreed to': 1,
 'Motion to Table Agreed to': 1,
 'Amendment Agreed to': 1,
 'Motion for Attendance Agreed to': 1,
 'Cloture Motion Rejected': 0,
 'Cloture Motion Agreed to': 1,
 'Cloture on the Motion to Proceed Rejected': 0,
 'Nomination Confirmed': 1,
 'Motion to Table Failed': 0,
 'Motion to Proceed Agreed to': 1,
 'Joint Resolution Passed': 1,
 'Veto Sustained': 0,
 'Motion Rejected': 0,
 'Concurrent Resolution Agreed to': 1,
 'Motion Agreed to': 1,
 'Conference Report Agreed to': 1,
 'Resolution Agreed to': 1,
 'Veto Overridden': 1}

df = df.replace(outcome_replacements)

In [632]:
#save clean
df.to_csv('cleaned_votes.csv')

#### Bill info

In [637]:
#get the bills data
url = "https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_114_2.xml"
req = urllib.request.Request(url)
with urllib.request.urlopen(req) as response:
    the_page = response.read()
root1 = ET.fromstring(the_page)

In [560]:
url = "https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_114_1.xml"
req = urllib.request.Request(url)
with urllib.request.urlopen(req) as response:
    the_page = response.read()
root2 = ET.fromstring(the_page)

In [566]:
for child in root1:
    print(child.tag, ":", child.text)

congress : 114
session : 2
congress_year : 2016
votes : 
    


In [588]:
bills = {}
for vote in root1[-1]:
    pieces = {}
    bills[vote[0].text] = pieces
    for element in vote:
        pieces[element.tag] = element.text
    pieces['yea'] = (vote[-2][0].text)
    pieces['nay'] = (vote[-2][1].text)
        


bills2 = {}
for vote in root2[-1]:
    pieces = {}
    bills2[vote[0].text] = pieces
    for element in vote:
        pieces[element.tag] = element.text
    pieces['yea'] = (vote[-2][0].text)
    pieces['nay'] = (vote[-2][1].text)
        

In [591]:
bills_114_2 = pd.DataFrame(bills).T.drop('vote_tally', axis=1)
bills_114_1 = pd.DataFrame(bills2).T.drop('vote_tally', axis=1)

In [638]:
bills_114_1.sample(10)

,issue,nay,question,result,title,vote_date,vote_number,yea
00254,H.R. 22,26,On the Cloture Motion,Agreed to,Motion to Invoke Cloture on Kirk Amdt. No. 232...,26-Jul,00254,67
00025,S. 1,38,On the Motion to Table,Agreed to,Motion to Table Carper Amdt. No. 121; To impos...,22-Jan,00025,57
00153,S.Con.Res. 11,52,On the Motion,Rejected,Stabenow Motion to Instruct (Medicare Cuts) Re...,15-Apr,00153,45
00081,S.Con.Res. 11,43,On the Amendment,Agreed to,Fischer Amdt. No. 409; To establish a deficit-...,24-Mar,00081,56
00168,H.R. 1191,54,On the Amendment,Rejected,Barrasso Amdt. No. 1147; To require a certific...,29-Apr,00168,45
00215,H.R. 1735,25,On Passage of the Bill,Passed,H.R. 1735 As Amended; An act to authorize appr...,18-Jun,00215,71
00176,H.R. 1314,45,On Cloture on the Motion to Proceed,Rejected,Motion to Invoke Cloture on the Motion to Proc...,12-May,00176,52
00064,H.R. 240,43,On the Cloture Motion,Rejected,Motion to Invoke Cloture on the Motion to Insi...,02-Mar,00064,47
00140,H.R. 2,57,On the Motion,Rejected,Motion to Waive Applicable Budgetary Disciplin...,14-Apr,00140,43
00148,S.Con.Res. 11,0,On the Motion,Agreed to,Burr Motion to Instruct (Student Loans) Re: S....,15-Apr,00148,97


In [596]:
new_index = []
for val in bills_114_2.index.values:
    i = int(val)
    new_num = i+339
    new_index.append(new_num)

In [598]:
new_index
bills_114_2.index = new_index

In [600]:
bills_114_2.head()

,issue,nay,question,result,title,vote_date,vote_number,yea
340,PN11,6,On the Nomination,Confirmed,"Nomination Luis Felipe Restrepo, of Pennsylvan...",11-Jan,00001,82
341,S. 2232,44,On Cloture on the Motion to Proceed,Rejected,Motion to Invoke Cloture on the Motion to Proc...,12-Jan,00002,53
342,PN367,36,On the Nomination,Confirmed,"Confirmation Wilhelmina Marie Wright, of Minne...",19-Jan,00003,58
343,H.R. 4038,43,On the Cloture Motion,Rejected,Motion to Invoke Cloture on the Motion to Proc...,20-Jan,00004,55
344,S.J.Res. 22,40,On the Cloture Motion,Rejected,Motion to Invoke Cloture on the Veto Message t...,21-Jan,00005,52


In [604]:
all_bills = bills_114_1.append(bills_114_2)

In [605]:
all_bills.sample(10)

,issue,nay,question,result,title,vote_date,vote_number,yea
00265,H.J.Res. 61,42,On the Cloture Motion,Rejected,Motion to Invoke Cloture on McConnell Amdt. No...,15-Sep,00265,56
00308,S. 1177,6,On the Cloture Motion,Agreed to,Motion to Invoke Cloture on the Motion to Disa...,18-Nov,00308,91
446,H.R. 2578,42,On the Motion to Table,Agreed to,Motion to Table the Motion to Commit with Inst...,20-Jun,00107,56
00235,S. 1177,64,On the Amendment,Rejected,Lee Amdt No. 2162; To amend the Elementary and...,14-Jul,00235,32
342,PN367,36,On the Nomination,Confirmed,"Confirmation Wilhelmina Marie Wright, of Minne...",19-Jan,00003,58
00162,S. 178,2,On the Amendment,Agreed to,Kirk Amdt. No. 273 As Modified; To amend title...,22-Apr,00162,97
00258,H.R. 22,38,On the Amendment,Agreed to,McConnell Amdt. No. 2266 As Amended; In the na...,29-Jul,00258,62
00176,H.R. 1314,45,On Cloture on the Motion to Proceed,Rejected,Motion to Invoke Cloture on the Motion to Proc...,12-May,00176,52
415,H.R. 2577,28,On the Motion,Agreed to,Motion to Waive All Applicable Budgetary Disci...,19-May,00076,70
00185,PN19,0,On the Nomination,Confirmed,"Confirmation Jose Rolando Olvera, Jr., of Texa...",21-May,00185,100


In [606]:
all_bills.to_csv('all_bills.csv')